# Getting started with ibmdbpy - Part 1: Basics

This notebook showcases ibmdbpy's key abstractions and features. It provides you with step-by-step examples to get started with the package.

___

### Accelerate Python analytics with in-database processing by using ibmdbpy and IBM Db2 Warehouse
 
The ibmdbpy project provides access to in-database algorithms in IBM Db2 Warehouse through a Python interface for data manipulation. It accelerates Python analytics by seamlessly pushing operations written in Python into the underlying database for execution, thereby benefitting from in-database performance-enhancing features, such as columnar storage and parallel processing. For more details about ibmdbpy, please refer to the [documentation](https://pythonhosted.org/ibmdbpy/index.html) and to the dedicated [Git repository](https://github.com/ibmdbanalytics/ibmdbpy/tree/master/ibmdbpy). This notebook provides you with an overview of ibmdbpy functionalities. 

 
__Prerequisites__
* Db2 account: see [IBM Cloud](https://cloud.ibm.com/login) or [Db2 Warehouse](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.kc.doc/welcome.html)
* Db2 driver: learn more on [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/en/SSFMBX/com.ibm.swg.im.dashdb.doc/connecting/connect_applications_by_type.html) and see [IBM Support](https://www.ibm.com/support/pages/db2-jdbc-driver-versions-and-downloads)
* Having installed the [ibmdbpy](https://pypi.org/project/ibmdbpy/) python library with pip: 
> pip install ibmdbpy 
* Optional dependency for JDBC is the [jaydebeapi](https://pypi.org/project/JayDeBeApi/) library. Run the following command to install ibmdbpy, as well as the dependencies for the JDBC feature:
> pip install ibmdbpy[jdbc]

__Contents__

1. Establish connection to Db2 database
2. Manipulate IdaDataFrames
3. Apply Machine Learning techniques with ibmdbpy

__Imports__

In [1]:
from ibmdbpy import IdaDataBase, IdaDataFrame
import pandas as pd

## 1. Establish connection to Db2 database

Two options are available to connect to a remote Db2 instance: ODBC and JDBC.
ODBC (Open Database Connectivity) is a standard API for accessing a database, independently of DBMS and OS.
JDBC (Java Database Connectivity) is an API for Java which defines how a client may access a database.
The JDBC Connection is based on a Java Virtual Machine, so it is available on every machine that can run Java. 

__ODBC__

Download an IBM DB2 driver and set up an ODBC connection, including the connection protocol, the port, and the host name, before establishing the connection. You find detailed information on the IBM Knowledge Center:

* ODBC for [IBM Db2 Warehouse](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.doc/connecting/connect_applications_by_type.html)
* ODBC for [IBM Db2 on Cloud](https://www.ibm.com/support/knowledgecenter/en/SSFMBX/com.ibm.swg.im.dashdb.doc/connecting/connect_applications_by_type.html) 

__JDBC__

This is how we will connect in this notebook. First, you need to dowload a valid driver (more info on [IBM Support](https://www.ibm.com/support/pages/db2-jdbc-driver-versions-and-downloads)). Then you need to put the `db2jcc.jar` or `db2jcc4.jar` file in the ibmdbpy site-package folder. When ibmdbpy runs, it checks whether one of those files exists in its installation folder and uses it to establish the connection. 

More details on IBM Knowledge Center:
* JDBC for [IBM Db2 Warehouse](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.doc/connecting/connect_connecting_jdbc_applications.html)
* JDBC for [IBM Db2 on Cloud](https://www.ibm.com/support/knowledgecenter/en/SSFMBX/com.ibm.swg.im.dashdb.doc/connecting/connect_connecting_jdbc_applications.html)

In [2]:
#Enter the values for you database connection
dsn_database = "___" # e.g. "BLUDB"
dsn_hostname = "___" # e.g.: "abc.url.example"
dsn_port = "___"    # e.g. "50000"
dsn_uid = "___"     # e.g. "db2_1234"
dsn_pwd = "___"     # e.g. "zorglub"

In [3]:
connection_string='jdbc:db2://'+dsn_hostname+':'+dsn_port+'/'+dsn_database+':user='+dsn_uid+';password='+dsn_pwd+";" 
# connect to the database, the idadb object holds the connection
idadb=IdaDataBase(dsn=connection_string)

Congratulations! You successfully connected to Db2 with ibmdbpy! When you are done, use `idadb.close()` to close the connection. To reconnect, or if the connection was broken, just use `idadb.reconnect()`. 

__Verbosity and autocommit__

The verbose mode automatically prints all SQL-communication between ibmdbpy and Db2, which can be very useful for debugging or understanding how ibmdbpy works. Choose the mode with `set_verbose()` or by setting the `verbosity` option when defining the IdaDataBase object. We encourage you to take a look at the prints in the first place, then feel free to silence the verbose. Note that printing adds delay when running cells.  

In [4]:
# Verbosity
from ibmdbpy.utils import set_verbose
set_verbose(False) # set to True to see the detail of the operations performed by ibmdbpy

By default the environment variable `AUTOCOMMIT` is set to True, which means that every SQL statement which is submitted through the connection is executed within its own transaction and then committed implicitly. When you close the connection to Db2, if the environment variable `AUTOCOMMIT` is set to False, then all changes after the last explicit commit are discarded. 

Let's get to it!

## 2. Manipulate IdaDataFrames

Let's explore ibmdbpy functionalities by using sample data. The well-known iris, swiss, and titanic datasets are furnished with the package.

__Load sample data__

In the following cells we first import the iris dataset as a pandas dataframe, then create an IdaDataFrame by creating a table `IRIS` into Db2 on the basis of this dataset. The option `clear_existing` is set to True, meaning that if a table called `IRIS` already exists in your database, it will be overwritten.

In [5]:
# Import sample data
from ibmdbpy.sampledata import iris

In [6]:
# Load the data into Db2, data is handled as an IdaDataFrame
# Note : clear_existing is set to True, which means that if a table named IRIS is already present in your database,
# it will we deleted so that a new one, defined below, is created.

idadf = idadb.as_idadataframe(iris, "IRIS", clear_existing = True)

print("Table written")

Uploading 150 rows (maxnrow was set to 20000)
Table written


Alternatively, if the table you want to manipulate has already been written into Db2, you can create an IdaDataFrame that directly points to this table. You need to specify the IdaDataBase object when creating an IdaDataFrame because this object holds the connection to the database.

In [7]:
# If table already exists in Db2
idadf_bis = IdaDataFrame(idadb, 'IRIS')

__Explore the data__

Ibmdbpy has been written in such a way that it is intuitive to use when you are already familiar with Pandas and Scikit-learn. For example you have the `head()` and `tail()` methods like in Pandas to take a look at the data. 

<font color='blue'>But what does ibmdbpy actually do?</font>
Ibmdbpy translates Pandas-like syntax into SQL and uses a middleware API (e.g. JayDeBeApi) to send it to an ODBC or JDBC-connected database for execution. After fetching the results, printing them will look similar to printing a pandas.DataFrame or a pandas.Series. Here you can see that data is indeed displayed like a Pandas dataframe.

In [8]:
# Take a look at the first records
idadf.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.3,3.0,1.1,0.1,setosa
1,4.4,3.0,1.3,0.2,setosa
2,4.4,2.9,1.4,0.2,setosa
3,4.4,3.2,1.3,0.2,setosa
4,4.5,2.3,1.3,0.3,setosa


In [9]:
# Take a look at the last records
idadf.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
145,7.7,3.0,6.1,2.3,virginica
146,7.7,2.6,6.9,2.3,virginica
147,7.7,3.8,6.7,2.2,virginica
148,7.7,2.8,6.7,2.0,virginica
149,7.9,3.8,6.4,2.0,virginica


Note: You can see in the two previous cells that the head and tail functions sort the data by the first numerical column. This can be disabled by setting the option `sort` to False when calling them. Other functions do not automatically sort the data.

__Simple statistics__

In this section you get a tour of the tools you use all the time for basic statistics.

In [10]:
# Display shape of the IDA dataframe
idadf.shape

(150, 5)

In [11]:
# Print column names
print([name for name in idadf.columns])

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']


In [12]:
# Correlation matrix: Here you can see that the output is a Pandas Dataframe.
corr = idadf.corr()
print(type(corr))

<class 'pandas.core.frame.DataFrame'>


In [13]:
# Display the correlation matrix:
corr

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.117570,0.871754,0.817941
sepal_width,-0.117570,1.000000,-0.428440,-0.366126
petal_length,0.871754,-0.428440,1.000000,0.962865
petal_width,0.817941,-0.366126,0.962865,1.000000


Notice that only numerical features have been taken into account in the correlation matrix above, like in Pandas. The column containing species names does not appear above because it has a `VARCHAR` data type. Same for the covariance matrix.

In [14]:
idadf.dtypes

,TYPENAME
sepal_length,DOUBLE
sepal_width,DOUBLE
petal_length,DOUBLE
petal_width,DOUBLE
species,VARCHAR


In [15]:
# Compute the covariance matrix
idadf.cov()

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,0.685694,-0.042434,1.274315,0.516271
sepal_width,-0.042434,0.189979,-0.329656,-0.121639
petal_length,1.274315,-0.329656,3.116278,1.295609
petal_width,0.516271,-0.121639,1.295609,0.581006


__Data selection and projection__

When working with an IdaDataFrame, you can use classic slicing like with NumPy arrays and Pandas DataFrames.

*Note: Use of an indexer*

An error would be raised by this line of code.

> idadf_new1 = idadf[0:9] 

We want to select the first ten rows but as the error message would say: "Row order is not guaranteed if no indexer was given and the dataset was not sorted". 
For ibmdbpy, using row indices only makes sense if an ID column is provided. Otherwise, the selection is non-deterministic. Here an index has not been provided so far, that is why an error would be produced. So let's use `add_column_id` to add an indexer column to the table. 

This way we really obtain a Pandas-like behavior and do some data selection and projection based on indices. If the `destructive` option is set to True, then this column will be physically added to the Db2 table and it will have a determinisic behavior. Otherwise, the physical table is not modified and each time you will print rows from the IdaDataFrame, row IDs will be assigned in a non-deterministic way. So we set `destructive = True` here.

In [16]:
# Here with an indexer
idadb.add_column_id(idadf, destructive=True)

# Now the idadataframe has the "ID" column set as indexer.
print("")
print("The idadataframe now has an indexer. Its name is %s." %idadf.indexer)
idadf.head()


The idadataframe now has an indexer. Its name is ID.


,ID,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa
2,2,4.7,3.2,1.3,0.2,setosa
3,3,4.6,3.1,1.5,0.2,setosa
4,4,5.0,3.6,1.4,0.2,setosa


The table has been physically modified. If you are using a Db2 table which already has an eligible ID column, you can set this column as indexer when defining the IdaDataFrame:
> idadf = IdaDataFrame(idadb, "DB2_TABLE_NAME", indexer = "ID_COLUMN_NAME")
    

We are ready to perform selection and projection.

In [17]:
### Selection ###

# no warning : we have an indexer
idadf[10:12].head()

,ID,sepal_length,sepal_width,petal_length,petal_width,species
0,10,5.4,3.7,1.5,0.2,setosa
1,11,4.8,3.4,1.6,0.2,setosa
2,12,4.8,3.0,1.4,0.1,setosa


In [18]:
# It is equivalent to the following using .loc
idadf_new2 = idadf.loc[10:12]

print(type(idadf_new2))
# Note: The output is still an IdaDataFrame here (a whole table like a Pandas DataFrame), 
# not an IdaSeries (single column like a Pandas Series).

idadf_new2.head()

<class 'ibmdbpy.frame.IdaDataFrame'>


,ID,sepal_length,sepal_width,petal_length,petal_width,species
0,10,5.4,3.7,1.5,0.2,setosa
1,11,4.8,3.4,1.6,0.2,setosa
2,12,4.8,3.0,1.4,0.1,setosa


In [19]:
### Projection ###

# Select columns
idadf_new3 = idadf[['sepal_length', 'sepal_width']]
print(type(idadf_new3))
idadf_new3.head()

<class 'ibmdbpy.frame.IdaDataFrame'>


,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


__Sorting__

You can sort rows with one or more columns as sorting criterion.

In [20]:
# One column
idadf_new = idadf.sort("sepal_length")
idadf_new.head()

,ID,sepal_length,sepal_width,petal_length,petal_width,species
0,13,4.3,3.0,1.1,0.1,setosa
1,38,4.4,3.0,1.3,0.2,setosa
2,8,4.4,2.9,1.4,0.2,setosa
3,42,4.4,3.2,1.3,0.2,setosa
4,41,4.5,2.3,1.3,0.3,setosa


In [21]:
# Several columns
idadf_new = idadf.sort(["sepal_length","sepal_width"])
idadf_new.head()

,ID,sepal_length,sepal_width,petal_length,petal_width,species
0,13,4.3,3.0,1.1,0.1,setosa
1,8,4.4,2.9,1.4,0.2,setosa
2,38,4.4,3.0,1.3,0.2,setosa
3,42,4.4,3.2,1.3,0.2,setosa
4,41,4.5,2.3,1.3,0.3,setosa


In [22]:
# Sort by columns using axis option
# 0 for rows, 1 for columns
idadf.sort(axis = 1, inplace=True)
idadf.head()

# by default, rows are sorted by ID

,ID,petal_length,petal_width,sepal_length,sepal_width,species
0,0,1.4,0.2,5.1,3.5,setosa
1,1,1.4,0.2,4.9,3.0,setosa
2,2,1.3,0.2,4.7,3.2,setosa
3,3,1.5,0.2,4.6,3.1,setosa
4,4,1.4,0.2,5.0,3.6,setosa


__Filtering__

You can filter rows with the usual comparison syntax.

In [23]:
idadf_new = idadf[idadf['species'] == 'versicolor']
idadf_new.head()

,ID,petal_length,petal_width,sepal_length,sepal_width,species
0,50,4.7,1.4,7.0,3.2,versicolor
1,51,4.5,1.5,6.4,3.2,versicolor
2,52,4.9,1.5,6.9,3.1,versicolor
3,53,4.0,1.3,5.5,2.3,versicolor
4,54,4.6,1.5,6.5,2.8,versicolor


In [24]:
idadf_new = idadf[idadf['sepal_length'] < 5]
idadf_new.head()

,ID,petal_length,petal_width,sepal_length,sepal_width,species
0,1,1.4,0.2,4.9,3.0,setosa
1,2,1.3,0.2,4.7,3.2,setosa
2,3,1.5,0.2,4.6,3.1,setosa
3,6,1.4,0.3,4.6,3.4,setosa
4,8,1.4,0.2,4.4,2.9,setosa


__Feature engineering__

Again, very Pandas-like! Enjoy the simplicity of using a one-line code which corresponds to a complex SQL query. Translation is done by ibmdbpy!

In [25]:
# Create a new column by multiplying two existing columns
idadf['new_col'] = idadf['sepal_length'] * idadf['sepal_width']
idadf.head()

,ID,petal_length,petal_width,sepal_length,sepal_width,species,new_col
0,0,1.4,0.2,5.1,3.5,setosa,17.85
1,1,1.4,0.2,4.9,3.0,setosa,14.70
2,2,1.3,0.2,4.7,3.2,setosa,15.04
3,3,1.5,0.2,4.6,3.1,setosa,14.26
4,4,1.4,0.2,5.0,3.6,setosa,18.00


In [26]:
# Replace existing column by newly computed values 
idadf['sepal_length'] /=2 
idadf.head()

,ID,petal_length,petal_width,sepal_length,sepal_width,species,new_col
0,0,1.4,0.2,2.55,3.5,setosa,17.85
1,1,1.4,0.2,2.45,3.0,setosa,14.70
2,2,1.3,0.2,2.35,3.2,setosa,15.04
3,3,1.5,0.2,2.30,3.1,setosa,14.26
4,4,1.4,0.2,2.50,3.6,setosa,18.00


__Saving your work__

Save your work in a new Db2 table. If you look for the IRIS table on Db2 Warehouse console, you will see that an ID column has been indeed added... but the sorting and projections and operations we performed above have not been written to the database! This changes only appear on the IdaDataFrame and will be lost once the connection is closed. So if you want to save your work, you need to either overwrite the original Db2 table or create a new table with a new name.

To overwrite the `IRIS` table:
> idadf.save_as(tablename = "IRIS", clear_existing = True)

To create a new table:
> idadf.save_as(tablename = "NEW_NAME", clear_existing = True)

In [27]:
idadf.save_as(tablename = "TEST", clear_existing = True)
# note: we have set clear_existing to True in case you run the notebook several times

/Users/eva.feillet/anaconda3/lib/python3.7/site-packages/ibmdbpy/frame.py:1899: UserWarning: Table TEST already exists.
  warnings.warn(message, UserWarning)


Are you sure that you want to overwrite TEST [y/n] y


__Wrap-up__

Nice job! You are now familiar with operations on IdaDataFrames with ibmdbpy. Let's go to the next section, where you see how to use ibmdbpy's machine learning functionalities.

## Apply Machine Learning techniques with ibmdbpy

Not only does ibmdbpy provide you with in-database Pandas-like functionalities, but also with in-database Scikit-learn-like functionalities. Algorithms dedicated to KMeans clustering, association-rule mining and Naive Bayes classification are provided in `ibmdbpy.learn`.

__Example: KMeans algorithm__

In [28]:
from ibmdbpy.learn import KMeans
kmeans = KMeans(3) # configure clustering with 3 cluters

Note: In-DataBase Kmeans needs an indexer to identify each row. As checked above this is the case for our IdaDataFrame. We can go ahead with model fitting and prediction!

In [29]:
# Taking a fresh new start
from ibmdbpy.sampledata import iris
idadf = idadb.as_idadataframe(iris, "IRIS", clear_existing = True)

# Reminder: an indexer must have been set for using .fit
idadb.add_column_id(idadf, destructive=True)

# here we check that we have an indexer
print(idadf.indexer)

Uploading 150 rows (maxnrow was set to 20000)
ID


In [30]:
# Model fitting and prediction 
kmeans.fit(idadf[['ID', 'sepal_length','sepal_width','petal_length','petal_width']])
pred = kmeans.predict(idadf)
pred.head()
# distance : distance to respective cluster center

,ID,CLUSTER_ID,DISTANCE
0,0,2,0.141351
1,1,2,0.447638
2,2,2,0.417109
3,3,2,0.525338
4,4,2,0.188627


By calling the `describe` method, you access the characteristics of the model which you have just build with `fit`. Here you see the coordinates (values) of the cluster centers in the K-Means model. For example, the first cluster center's petals have a length of 5.549020 units.

In [31]:
kmeans.describe()

KMeans clustering with 3 clusters of sizes 37, 63, 50

Cluster means: 
   CLUSTERID  sepal_length  sepal_width  petal_length  petal_width
0          1      6.601587     2.985714      5.384127     1.915873
1          2      5.006000     3.428000      1.462000     0.246000
2          3      5.683784     2.678378      4.091892     1.267568

Within cluster sum of squares by cluster:
[16.57416896 53.22701047 15.151     ]


Take a closer look at the predictions:

In [32]:
iris_df = idadf.as_dataframe().set_index('ID')
pred_df = pred.as_dataframe().set_index('ID')
merged_df = iris_df.merge(pred_df, left_index=True, right_index=True)
merged_df.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species,CLUSTER_ID,DISTANCE
ID,,,,,,,
0,5.1,3.5,1.4,0.2,setosa,2,0.141351
1,4.9,3.0,1.4,0.2,setosa,2,0.447638
2,4.7,3.2,1.3,0.2,setosa,2,0.417109
3,4.6,3.1,1.5,0.2,setosa,2,0.525338
4,5.0,3.6,1.4,0.2,setosa,2,0.188627


Of course there are a few misclassified examples, but the original classes have been broadly rediscovered. Label 1 corresponds to the virginica specy, label 2 to setosa and label 3 to versicolor.

In [33]:
# overview
pd.crosstab(merged_df['CLUSTER_ID'], merged_df['species'])

species,setosa,versicolor,virginica
CLUSTER_ID,,,
1,0,12,47
2,50,0,0
3,0,38,3


__Other ML algorithms__

Ibmdbpy also provides you with tools for association rules mining and naive bayes classification.

For more details, see the separate notebooks about machine learning with ibmdbpy on our GitHub repository!

___

__Final step: Close the connection__

Closing the IdaDataBase is equivalent to closing the connection: once the connection is closed, it is not possible to use the IdaDataBase instance and any IdaDataFrame instances that were opened on this connection anymore.

In [34]:
idadb.close()
# to reconnect: idadb.reconnect()

Connection closed.


## Where to go from here ?

Well done! You are now familiar with the basic functionalities of ibmdbpy! You are ready to explore the other notebooks of this series:

* Getting started with ibmdbpy geospatial functions:
        
    [GeoBasics](./ibmdbpy_GettingStarted_2-geo_basics.ipynb)

    [Extensive guide](./ibmdbpy_GettingStarted_3-geo_guide.ipynb)
    

* Ibmdbpy in practice : analyze the Museums dataset, understand how to create IdaDataFrames and IdaGeoDataFrame:
        
    [Preprocessing](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_1-preprocessing.ipynb)

    [Geospatial recommendation](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_2-geospatial.ipynb)


* Machine learning with ibmdbpy: 
        
    [Naïve Bayes](../MachineLearning/ibmdbpy_NaiveBayes.ipynb)

    [Association Rules Mining](../MachineLearning/ibmdbpy_AssociationRulesMining.ipynb)

____

__Authors__

Eva Feillet - ML intern, IBM Cloud and Cognitive Software @ IBM Lab in Böblingen, Germany